## TF-IDF 벡터화와 유사도 구해보기

In [1]:
docs = [
  '먹고 싶은 사과', # 문서0 
  '먹고 싶은 바나나', # 문서1
  '길고 노란 바나나 바나나', # 문서2 
  '저는 과일이 좋아요' # 문서3 
]

In [2]:
docs

['먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요']

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # Counter Vectorizer 객체 생성

In [5]:
vect

CountVectorizer()

In [6]:
# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(docs) 
countvect # 4x9 : 4개의 문서에 9개의 단어 

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [7]:
# toarray()를 통해서 문장이 Vector 형태의 값을 얻을 수 있음 
# 하지만, 각 인덱스와 컬럼이 무엇을 의미하는지에 대해서는 알 수가 없음 
# sparse matrix -> numpy
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [8]:
vect.vocabulary_  # 몇번째에 존재하는지 알 수 있음. 순서가 뒤죽박죽

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [9]:
# sorted라는 함수를 통해서 단어를 정렬
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [10]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [11]:
# sklearn이 정말 잘 만들어져있음.
# 위의 Data Frame 형태의 유사도를 계산 
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)
# 문서 1을 본 사람들에게 문서 2를 추천
# 문서 2를 본 사람들에게 문서 1과 문서 3을 추천할 수 있다.

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [13]:
# CountVectorizer -> TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

In [14]:
tfvect

TfidfVectorizer()

In [15]:
tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidv_df, tfidv_df)
# countvectorizer를 이용했을 때와는 값이 다르다. 실제 소프트웨어의 계산방식이 조금은 다를 수도 있기 때문.

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [17]:
# 파라미터를 어떻게 적용하느냐에 따라서도 값이 달라질 수 있다. 4개로 제한
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=4)
tfvect = vect.fit(docs)

In [18]:
tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,먹고,바나나,싶은
문서1,0.0,0.707107,0.00000,0.707107
문서2,0.0,0.577350,0.57735,0.577350
문서3,0.0,0.000000,1.00000,0.000000
문서4,1.0,0.000000,0.00000,0.000000


## 실전 예제 (영화 추천)

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# import os
# print(os.listdir("../projects_ys/Recommender System/data/movies/"))

In [ ]:
# # 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
# path = '../projects_ys/Recommender System/data/movies/'

In [21]:
# IMDB(영화 평점 사이트)에서 가져온 영화의 메타데이터
data = pd.read_csv('./data/movies/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
# overview의 항목 추출 
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [23]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

(44512, 24)

In [25]:
# 램 용량이 부족할 수 있기 대문에, 10000개의 데이터만 활용
data = data.loc[0:10000].reset_index(drop=True) # drop=True는 기존 인덱스 제거

In [26]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
# https://wikidocs.net/22530
tfidf = TfidfVectorizer(stop_words='english')

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(10001, 32383)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생하신 분은 TF-IDF의 파라미터를 수정해줘서 다시 돌리면 됩니다. 
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행해보시길 바랍니다. 
# data = data.loc[0:10000].reset_index(drop=True)

In [28]:
cosine_matrix.shape

(10001, 10001)

In [29]:
np.round(cosine_matrix, 4)

# cosine-matrix
# 인덱스 0 -> 1 -> 2 ... -> 10001
# 0 인덱스 : 0번째 id 영화
# 즉, 인덱스와 영화 Id가 같지 않아, mapping을 해줄 필요가 있다.

array([[1.    , 0.0168, 0.    , ..., 0.    , 0.    , 0.    ],
       [0.0168, 1.    , 0.0487, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.0487, 1.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 1.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 1.    ]])

In [31]:
# 편의상 인덱스가 아닌 title과 id를 매핑한다.

# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [30]:
# enumerate에 대하여..
for i, c in enumerate(data['title']):
    print(i)
    print(c)
    break # 한번만 돌고 멈추도록 한다.

0
Toy Story


첫번째 영화인 Toy Story에 대해서 유사도를 구해보자.

In [37]:
# items => key와 value 모두 호출해준다.
# movie2id.items()

In [39]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
# sim_scores

In [40]:
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(2979, 0.4565298021774398),
 (8303, 0.21256230971004703),
 (1058, 0.18335909389518892),
 (3039, 0.14790265373244732),
 (1916, 0.14191688958568915),
 (483, 0.1412700228156645),
 (5776, 0.13662456333307288),
 (7230, 0.13021542285694832),
 (6920, 0.12632401336138635),
 (7591, 0.1246393954601935)]

In [41]:
# 인덱스에 해당되는 title이 무엇인지 모르기 때문에 아까 만들어둔 movie2id 딕셔너리에 적용해준다
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 2', 0.4565298021774398),
 ('The Champ', 0.21256230971004703),
 ('Rebel Without a Cause', 0.18335909389518892),
 ('Man on the Moon', 0.14790265373244732),
 ('Condorman', 0.14191688958568915),
 ('Malice', 0.1412700228156645),
 ('Class of 1984', 0.13662456333307288),
 ('Africa Screams', 0.13021542285694832),
 ('Rivers and Tides', 0.12632401336138635),
 ('The First $20 Million Is Always the Hardest', 0.1246393954601935)]

**Toy story overview 기준으로 TF-IDF 정렬한 결과 Toy Story2가 유사함을 알 수 있다**